<a href="https://colab.research.google.com/github/aniketSanyal/DifferentialPrivacy/blob/main/Prefix_DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:

class Config:
  num_train_epochs = 4
  learning_rate = 1e-3
  n_prompt_tokens = 10
  random_range  = 0.5
  batch_size = 64
  max_grad_norm = 0.1
args = Config()


In [26]:
import torch
from torch.nn.utils.rnn import pad_sequence

def custom_collate(batch):
    input_ids = pad_sequence([item['input_ids'] for item in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence([item['attention_mask'] for item in batch], batch_first=True, padding_value=0)
    labels = torch.stack([item['labels'] for item in batch])

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

In [67]:
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType


from datasets import load_dataset
raw_datasets = load_dataset("glue", "sst2")


from transformers import AutoTokenizer

checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, num_labels=2)


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["sentence"], max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")



train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)


In [68]:

import numpy as np

def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()

    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:
        #batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss, logits = outputs[:2]

        preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
        labels =batch['labels'].detach().cpu().numpy()

        loss_arr.append(loss.item())
        accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

In [69]:
LOGGING_INTERVAL = 100

In [70]:
def train_prefix(model,optimiser, train_dataloader):
  for epoch in range(1, args.num_train_epochs+1):
    losses = []

    for  step, batch in enumerate(train_dataloader):
            optimiser.zero_grad()
            #batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch) # output = loss, logits, hidden_states, attentions
            loss = outputs[0]

            loss.backward()
            losses.append(loss.item())
            optimiser.step()

            if step > 0 and step % LOGGING_INTERVAL == 0:
                train_loss = np.mean(losses)
                eps = privacy_engine.get_epsilon(DELTA)
                eval_loss, eval_accuracy = evaluate(model)

                print(
                  f"Epoch: {epoch} | "
                  f"Step: {step} | "
                  f"Train loss: {train_loss:.3f} | "
                  f"Eval loss: {eval_loss:.3f} | "
                  f"Eval accuracy: {eval_accuracy:.3f} | "
                  f"ɛ: {eps:.2f}"
                )

In [71]:
from transformers import  AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, num_virtual_tokens=10)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 5,378 || all params: 4,391,556 || trainable%: 0.12246228899278525


In [72]:
from torch.optim import AdamW

optimiser = AdamW(model.parameters(), lr=args.learning_rate)

In [73]:
DELTA = 1/len(train_dataloader)

In [74]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)


In [75]:
train_prefix(model, optimiser, train_dataloader)

Epoch: 1 | Step: 100 | Train loss: 1.660 | Eval loss: 2.521 | Eval accuracy: 0.509 | ɛ: 2.49
Epoch: 1 | Step: 200 | Train loss: 1.909 | Eval loss: 2.401 | Eval accuracy: 0.509 | ɛ: 3.17
Epoch: 1 | Step: 300 | Train loss: 1.972 | Eval loss: 2.175 | Eval accuracy: 0.509 | ɛ: 3.58
Epoch: 1 | Step: 400 | Train loss: 1.993 | Eval loss: 2.128 | Eval accuracy: 0.509 | ɛ: 3.88


KeyboardInterrupt: ignored

QNLI

In [37]:
raw_datasets = load_dataset("glue", "qnli")


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation'] = raw_datasets['validation'].select([i for i in range(5000)])

raw_datasets['test'] = raw_datasets['test'].select([i for i in range(5000)])


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["question"],example["sentence"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'sentence', 'question'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [54]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, num_virtual_tokens=10)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = get_peft_model(model, peft_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
optimiser = AdamW(model.parameters(), lr=args.learning_rate)

In [56]:
DELTA = 1/len(train_dataloader)

In [57]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)


/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)


In [58]:
train_prefix(model, optimiser, train_dataloader)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1324: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "


Epoch: 1 | Step: 100 | Train loss: 0.710 | Eval loss: 0.690 | Eval accuracy: 0.541 | ɛ: 2.62
Epoch: 1 | Step: 200 | Train loss: 0.721 | Eval loss: 0.698 | Eval accuracy: 0.542 | ɛ: 3.33
Epoch: 1 | Step: 300 | Train loss: 0.727 | Eval loss: 0.706 | Eval accuracy: 0.554 | ɛ: 3.76
Epoch: 1 | Step: 400 | Train loss: 0.747 | Eval loss: 0.724 | Eval accuracy: 0.554 | ɛ: 4.08
Epoch: 1 | Step: 500 | Train loss: 0.767 | Eval loss: 0.851 | Eval accuracy: 0.536 | ɛ: 4.35
Epoch: 1 | Step: 600 | Train loss: 0.793 | Eval loss: 0.803 | Eval accuracy: 0.553 | ɛ: 4.58
Epoch: 1 | Step: 700 | Train loss: 0.818 | Eval loss: 0.824 | Eval accuracy: 0.551 | ɛ: 4.79
Epoch: 2 | Step: 100 | Train loss: 1.082 | Eval loss: 0.921 | Eval accuracy: 0.547 | ɛ: 5.13
Epoch: 2 | Step: 200 | Train loss: 1.086 | Eval loss: 0.949 | Eval accuracy: 0.549 | ɛ: 5.30
Epoch: 2 | Step: 300 | Train loss: 1.093 | Eval loss: 0.893 | Eval accuracy: 0.565 | ɛ: 5.47
Epoch: 2 | Step: 400 | Train loss: 1.110 | Eval loss: 0.929 | Eval acc

KeyboardInterrupt: ignored

QQP

In [76]:
raw_datasets = load_dataset("glue", "qqp")


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation'] = raw_datasets['validation'].select([i for i in range(5000)])

raw_datasets['test'] = raw_datasets['test'].select([i for i in range(5000)])


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["question1"],example["question2"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'question1', 'question2'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [77]:
del model
del optimiser

In [78]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, num_virtual_tokens=10)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = get_peft_model(model, peft_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [79]:
optimiser = AdamW(model.parameters(), lr=args.learning_rate)

In [80]:
DELTA = 1/len(train_dataloader)

In [81]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)


/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)


In [82]:
train_prefix(model, optimiser, train_dataloader)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1324: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "


Epoch: 1 | Step: 100 | Train loss: 1.734 | Eval loss: 2.057 | Eval accuracy: 0.637 | ɛ: 2.62
Epoch: 1 | Step: 200 | Train loss: 1.850 | Eval loss: 2.045 | Eval accuracy: 0.637 | ɛ: 3.33
Epoch: 1 | Step: 300 | Train loss: 1.894 | Eval loss: 1.993 | Eval accuracy: 0.637 | ɛ: 3.76
Epoch: 1 | Step: 400 | Train loss: 1.915 | Eval loss: 2.001 | Eval accuracy: 0.637 | ɛ: 4.08
Epoch: 1 | Step: 500 | Train loss: 1.931 | Eval loss: 2.022 | Eval accuracy: 0.637 | ɛ: 4.35
Epoch: 1 | Step: 600 | Train loss: 1.940 | Eval loss: 1.969 | Eval accuracy: 0.637 | ɛ: 4.58
Epoch: 1 | Step: 700 | Train loss: 1.944 | Eval loss: 2.002 | Eval accuracy: 0.637 | ɛ: 4.79


KeyboardInterrupt: ignored

MNLI

In [83]:
raw_datasets = load_dataset("glue", "mnli")


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation_matched'] = raw_datasets['validation_matched'].select([i for i in range(5000)])



tokenizer = AutoTokenizer.from_pretrained(checkpoint, num_labels =3)


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["premise"],example["hypothesis"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'premise', 'hypothesis'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation_matched"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [84]:
del model
del optimiser

In [91]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, num_virtual_tokens=10)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
model = get_peft_model(model, peft_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [92]:
optimiser = AdamW(model.parameters(), lr=args.learning_rate)

In [93]:
DELTA = 1/len(train_dataloader)

In [94]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)

In [95]:
train_prefix(model, optimiser, train_dataloader)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1324: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "


Epoch: 1 | Step: 100 | Train loss: 1.111 | Eval loss: 1.100 | Eval accuracy: 0.343 | ɛ: 2.62
Epoch: 1 | Step: 200 | Train loss: 1.108 | Eval loss: 1.098 | Eval accuracy: 0.344 | ɛ: 3.33
Epoch: 1 | Step: 300 | Train loss: 1.106 | Eval loss: 1.101 | Eval accuracy: 0.354 | ɛ: 3.76
Epoch: 1 | Step: 400 | Train loss: 1.107 | Eval loss: 1.112 | Eval accuracy: 0.351 | ɛ: 4.08
Epoch: 1 | Step: 500 | Train loss: 1.107 | Eval loss: 1.099 | Eval accuracy: 0.348 | ɛ: 4.35
Epoch: 1 | Step: 600 | Train loss: 1.106 | Eval loss: 1.113 | Eval accuracy: 0.347 | ɛ: 4.58
Epoch: 1 | Step: 700 | Train loss: 1.106 | Eval loss: 1.117 | Eval accuracy: 0.342 | ɛ: 4.79
Epoch: 2 | Step: 100 | Train loss: 1.110 | Eval loss: 1.102 | Eval accuracy: 0.347 | ɛ: 5.13
Epoch: 2 | Step: 200 | Train loss: 1.108 | Eval loss: 1.110 | Eval accuracy: 0.339 | ɛ: 5.30
Epoch: 2 | Step: 300 | Train loss: 1.106 | Eval loss: 1.134 | Eval accuracy: 0.336 | ɛ: 5.47
Epoch: 2 | Step: 400 | Train loss: 1.106 | Eval loss: 1.100 | Eval acc